In [1]:
import pandas as pd

files = [f"/lakehouse/default/Files/Bronze_taxi/yellow_tripdata_2024-{m:02d}.parquet" for m in range(1, 13)]

use_cols = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "total_amount",
    "trip_distance",
    "fare_amount"
]

# 1) read taxi
df = pd.concat([pd.read_parquet(f, columns=use_cols) for f in files], ignore_index=True)

# 2) datetimes + basic cleaning
df["pickup_dt"]  = pd.to_datetime(df["tpep_pickup_datetime"], errors="coerce")
df["dropoff_dt"] = pd.to_datetime(df["tpep_dropoff_datetime"], errors="coerce")

df = df.dropna(subset=["pickup_dt"])
df["date"] = df["pickup_dt"].dt.date
df["hour"] = df["pickup_dt"].dt.hour
df["trip_duration_min"] = (df["dropoff_dt"] - df["pickup_dt"]).dt.total_seconds() / 60

# 3) strict date filter 
start = pd.Timestamp("2024-01-01")
end = pd.Timestamp("2025-01-01")  

df = df[(df["pickup_dt"] >= start) & (df["pickup_dt"] < end)].copy()

# 4) join zone lookup
zones = pd.read_csv("/lakehouse/default/Files/Bronze_taxi/taxi_zone_lookup.csv")

zones_p = zones[["LocationID", "Borough", "Zone"]].rename(columns={"LocationID": "PULocationID", "Zone": "pickup_zone", "Borough": "Borough_p"})
df = df.merge(zones_p, on="PULocationID", how="left")

zones_d = zones[["LocationID", "Borough", "Zone"]].rename(columns={"LocationID": "DOLocationID", "Zone": "destination_zone", "Borough": "Borough_d"})
df = df.merge(zones_d, on="DOLocationID", how="left")


# df = df[df["Borough"] == "Manhattan"].copy()


# silver table daily
silver_taxi_daily_zone = (
    df.groupby(["date", "Borough_p", "PULocationID"], as_index=False)
      .agg(
          trips_count=("date", "size"),
          total_revenue_usd=("total_amount", "sum"),
          avg_total_amount_usd=("total_amount", "mean"),
          avg_trip_distance=("trip_distance", "mean"),
          avg_duration_min=("trip_duration_min", "mean"),
      )
      .sort_values(["date"])
)

# silver table hourly
silver_taxi_hourly_zone = (
    df.groupby(["date", "hour", "Borough_p", "PULocationID"], as_index=False)
      .agg(
          trips_count=("date", "size"),
          avg_total_amount_usd=("total_amount", "mean"),
      )
      .sort_values(["date", "hour"])
)

# dim table zone
dim_taxi_zone = zones.copy()

silver_taxi_daily_zone.head()

StatementMeta(, c6f2a92d-bbe7-4147-af8a-55420d6f9f11, 3, Finished, Available, Finished)

,date,Borough_p,PULocationID,trips_count,total_revenue_usd,avg_total_amount_usd,avg_trip_distance,avg_duration_min
0,2024-01-01,Bronx,3,1,36.77,36.770000,9.660000,18.166667
147,2024-01-01,Manhattan,232,321,9000.41,28.038660,3.340623,16.277778
148,2024-01-01,Manhattan,233,857,22652.70,26.432555,3.122100,12.842396
149,2024-01-01,Manhattan,234,1694,37977.86,22.419044,2.365130,17.434662
150,2024-01-01,Manhattan,236,1763,36479.08,20.691480,2.251123,9.862554


In [ ]:
spark_silver_taxi_daily_zone = spark.createDataFrame(silver_taxi_daily_zone)


spark_silver_taxi_daily_zone.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_taxi_daily_zone")

print("Done process")

StatementMeta(, c6f2a92d-bbe7-4147-af8a-55420d6f9f11, 5, Finished, Available, Finished)

Done process


In [ ]:
trips_by_hour = (df.groupby("hour")
                 .size()
                 .reset_index(name="trips")
                 .sort_values("hour"))

revenue_by_day = (df.groupby("date")
                  .agg(
                      trips=("date", "size"),
                      total_revenue_usd=("total_amount", "sum"),
                      avg_fare_usd=("fare_amount", "mean"),
                      avg_total_usd=("total_amount", "mean"),
                      avg_distance=("trip_distance", "mean"),
                      avg_duration_min=("trip_duration_min", "mean")
                  )
                  .reset_index()
                  .sort_values("date"))

top_pickups = (df.groupby(["PULocationID", "Borough_p"])
               .size()
               .reset_index(name="trips")
               .sort_values("trips", ascending=False))

StatementMeta(, 924380d3-5e2b-46ea-9eb8-d3ed74267cb5, 5, Finished, Available, Finished)

In [ ]:
import pyspark.sql.functions as F
spark_daily = spark.createDataFrame(silver_taxi_daily_zone)
spark_hourly = spark.createDataFrame(silver_taxi_hourly_zone)
spark_dim = spark.createDataFrame(dim_taxi_zone)

gold_taxi_trips_by_hour = spark.createDataFrame(trips_by_hour)
gold_taxi_revenue_by_day = spark.createDataFrame(revenue_by_day)
gold_taxi_top_pickups = spark.createDataFrame(top_pickups)

gold_taxi_trips_by_hour.write.mode("append").saveAsTable("gold_taxi_trips_by_hour")
gold_taxi_revenue_by_day.write.mode("append").saveAsTable("gold_taxi_revenue_by_day")
gold_taxi_top_pickups.write.mode("append").saveAsTable("gold_taxi_top_pickups")

spark_daily.write.mode("append").saveAsTable("silver_taxi_daily_zone")
spark_hourly.write.mode("append").saveAsTable("silver_taxi_hourly_zone")
spark_dim.write.mode("append").saveAsTable("dim_taxi_zone")

StatementMeta(, 924380d3-5e2b-46ea-9eb8-d3ed74267cb5, 6, Finished, Available, Finished)

In [ ]:
df = spark.sql("SELECT * FROM Itransition_project.dbo.silver_taxi_daily_zone LIMIT 1000")
display(df)

StatementMeta(, c6f2a92d-bbe7-4147-af8a-55420d6f9f11, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0252b865-bc05-4a67-855b-d4d9ac2b0cf7)